In [1]:
# Added Label smoothing & AdamW and SE blocks and Slightly stronger SpecAugment
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers as L, models as M, callbacks as C
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

2025-09-10 15:08:57.574084: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-10 15:08:57.601945: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-10 15:08:58.210213: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
# =========================
#           PATHS
# =========================
DRONE_ROOT   = r"/home/destrox-907/Husnian_FYP/Dataset/MFCC Drone Dataset/MFCC Drone Dataset"
NODRONE_ROOT = r"/home/destrox-907/Husnian_FYP/Dataset/MFCC Noise Dataset/MFCC Noise Dataset"

# =========================
#      CONFIG / HYPERS
# =========================
INPUT_SHAPE     = (13, 40, 1)
BATCH_SIZE      = 128
EPOCHS          = 40
LEARNING_RATE   = 1e-3
MIXUP_ALPHA     = 0.3
USE_SPECAUG     = True
VAL_SIZE        = 0.20
SEED            = 42

# =========================
#  RECURSIVE DATA LOADING
# =========================
def collect_npy_paths(root_dir):
    paths = []
    for r, _, files in os.walk(root_dir):
        for f in files:
            if f.lower().endswith(".npy"):
                paths.append(os.path.join(r, f))
    return paths

def load_paths_and_labels(drone_root, nodrone_root):
    drone_paths   = collect_npy_paths(drone_root)
    nodrone_paths = collect_npy_paths(nodrone_root)
    paths  = np.array(drone_paths + nodrone_paths)
    labels = np.array([1]*len(drone_paths) + [0]*len(nodrone_paths), dtype=np.int32)
    return paths, labels

paths, labels = load_paths_and_labels(DRONE_ROOT, NODRONE_ROOT)
print(f"Total .npy files: {len(paths)} | drone={labels.sum()} | no_drone={len(labels)-labels.sum()}")

X_train, X_val, y_train, y_val = train_test_split(
    paths, labels, test_size=VAL_SIZE, random_state=SEED, stratify=labels
)

# =========================
#     DATA PIPELINE
# =========================
def npy_loader(path):
    try:
        arr = np.load(path.decode("utf-8")).astype("float32")
        arr = np.reshape(arr, (13, 40, 1))
        return arr
    except Exception as e:
        print(f"[WARN] Failed to load {path.decode('utf-8')}: {e}")
        return np.zeros((13, 40, 1), dtype="float32")

def tf_load(path, label, training=True):
    x = tf.numpy_function(npy_loader, [path], Tout=tf.float32)
    x = tf.ensure_shape(x, INPUT_SHAPE)

    if training and USE_SPECAUG:
        # >>> CHANGED: stronger SpecAugment
        def _specaug(a):
            a = a.copy()
            f_dim, t_dim, _ = a.shape  # 13,40,1
            for _ in range(3):
                f = np.random.randint(0, 5)      # up to 4 MFCC bins
                if f > 0:
                    f0 = np.random.randint(0, f_dim - f + 1)
                    a[f0:f0+f, :, :] = 0.0
            for _ in range(3):
                t = np.random.randint(0, 11)     # up to 10 time frames
                if t > 0:
                    t0 = np.random.randint(0, t_dim - t + 1)
                    a[:, t0:t0+t, :] = 0.0
            return a
        x = tf.numpy_function(_specaug, [x], Tout=tf.float32)
        x = tf.ensure_shape(x, INPUT_SHAPE)

    y = tf.one_hot(label, 2)
    return x, y

def make_dataset(paths, labels, batch=BATCH_SIZE, training=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        ds = ds.shuffle(8192, reshuffle_each_iteration=True)
    ds = ds.map(lambda p, l: tf_load(p, l, training=training),
                num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch)

    if training and MIXUP_ALPHA > 0.0:
        def mixup_batch(x, y):
            lam = tf.random.uniform([], 0.5, 1.0)
            idx = tf.random.shuffle(tf.range(tf.shape(x)[0]))
            x2 = tf.gather(x, idx); y2 = tf.gather(y, idx)
            return lam*x + (1-lam)*x2, lam*y + (1-lam)*y2
        ds = ds.map(mixup_batch, num_parallel_calls=tf.data.AUTOTUNE)

    return ds.prefetch(tf.data.AUTOTUNE)

train_ds = make_dataset(X_train, y_train, training=True)
val_ds   = make_dataset(X_val,   y_val,   training=False)


In [ ]:
# =========================
#   RESNET-LIKE BACKBONE (+SE)
# =========================
def squeeze_excite(x, r=8):
    c = x.shape[-1]
    s = L.GlobalAveragePooling2D()(x)
    s = L.Dense(max(c//r, 8), activation="relu")(s)
    s = L.Dense(c, activation="sigmoid")(s)
    s = L.Reshape((1,1,c))(s)
    return L.Multiply()([x, s])

def res_block(x, filters, stride=1, use_se=False):
    shortcut = x
    x = L.Conv2D(filters, (3,3), strides=stride, padding="same", use_bias=False)(x)
    x = L.BatchNormalization()(x); x = L.ReLU()(x)
    x = L.Conv2D(filters, (3,3), padding="same", use_bias=False)(x)
    x = L.BatchNormalization()(x)
    if use_se:
        x = squeeze_excite(x)
    if shortcut.shape[-1] != filters or stride != 1:
        shortcut = L.Conv2D(filters, (1,1), strides=stride, padding="same", use_bias=False)(shortcut)
        shortcut = L.BatchNormalization()(shortcut)
    x = L.Add()([x, shortcut]); x = L.ReLU()(x)
    return x

def build_model(input_shape=(13,40,1), num_classes=2):
    inp = L.Input(shape=input_shape)
    x = L.Conv2D(32, (3,3), padding="same", use_bias=False)(inp)
    x = L.BatchNormalization()(x); x = L.ReLU()(x)

    x = res_block(x, 32, use_se=True)
    x = res_block(x, 32, use_se=True)
    x = L.MaxPool2D(pool_size=(1,2))(x)

    x = res_block(x, 64, stride=2, use_se=True)
    x = res_block(x, 64, use_se=True)

    x = L.GlobalAveragePooling2D()(x)
    x = L.Dense(128, activation="relu")(x)
    x = L.Dropout(0.3)(x)
    out = L.Dense(num_classes, activation="softmax")(x)
    return M.Model(inp, out)

model = build_model()
model.summary()

In [ ]:
# =========================
#      TRAIN & EVAL
# =========================
# >>> CHANGED: Label smoothing + AdamW
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05)
opt  = tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=1e-4)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

cbs = [
    C.ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1),
    C.EarlyStopping(monitor="val_accuracy", patience=8, restore_best_weights=True),
    C.ModelCheckpoint("drone_resnet_best.keras", monitor="val_accuracy", save_best_only=True)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=cbs)

probs = model.predict(val_ds)[:, 1]
y_true = np.concatenate([y.numpy() for _, y in val_ds])[:, 1]
auroc = roc_auc_score(y_true, probs)
prec, rec, thr = precision_recall_curve(y_true, probs)
auprc = auc(rec, prec)
cands = np.linspace(0.05, 0.95, 19)
f1s = [f1_score(y_true, (probs >= t).astype(int)) for t in cands]
best_thr, best_f1 = cands[int(np.argmax(f1s))], max(f1s)


In [ ]:
# --- SAVE THE FINAL MODEL (whatever weights are in memory now) ---
model.save("drone_resnet_final_form.keras")

In [ ]:
print(f"\n=== Validation Metrics ===")
print(f"AUROC  : {auroc:.4f}")
print(f"AUPRC  : {auprc:.4f}")
print(f"Best F1: {best_f1:.4f} @ threshold={best_thr:.2f}")

y_pred = (probs >= best_thr).astype(int)
cm = confusion_matrix(y_true, y_pred, labels=[0,1])
print("Confusion Matrix (rows=true, cols=pred):")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Drone", "Drone"])
disp.plot(cmap=plt.cm.Blues, values_format="d")
plt.title("Confusion Matrix")
plt.show()

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["No Drone", "Drone"]))
